SentenceTransformers: https://sbert.net/

# Load Data (Without Reviews)

In [ ]:
import pandas as pd
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/totallymakescents/fra_standard.csv'
with open(file_path, encoding='latin1') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        print(f"Line {i+1}: {row}")
        if i >= 9:  # print first 10 lines
            break

Line 1: ['', 'url', 'Perfume', 'Brand', 'Rating Count', 'Top', 'Middle', 'Base', 'mainaccord1', 'mainaccord2', 'mainaccord3', 'mainaccord4', 'mainaccord5']
Line 2: ['0', 'https://www.fragrantica.com/perfume/xerjoff/accento-overdose-pride-edition-74630.html', 'Accento Overdose Pride Edition', 'Xerjoff', '201', 'fruity notes, aldehydes, green notes', 'bulgarian rose, egyptian jasmine, lily-of-the-valley', 'eucalyptus, pine', 'rose', 'woody', 'fruity', 'aromatic', 'floral']
Line 3: ['1', 'https://www.fragrantica.com/perfume/jean-paul-gaultier/classique-pride-2024-90394.html', 'Classique Pride 2024', 'Jean Paul Gaultier', '70', 'yuzu, citruses', 'orange blossom, neroli', 'musk, blonde woods', 'citrus', 'white floral', 'sweet', 'fresh', 'musky']
Line 4: ['2', 'https://www.fragrantica.com/perfume/jean-paul-gaultier/classique-pride-2023-81775.html', 'Classique Pride 2023', 'Jean Paul Gaultier', '285', 'blood orange, yuzu', 'neroli, orange blossom', 'musk, white woods', 'citrus', 'white floral

In [ ]:
df = pd.read_csv(
    file_path,
    sep=',',            # specify correct separator
    encoding='latin1',
    low_memory=False
)

In [ ]:
df.columns

Index(['Unnamed: 0', 'url', 'Perfume', 'Brand', 'Rating Count', 'Top',
       'Middle', 'Base', 'mainaccord1', 'mainaccord2', 'mainaccord3',
       'mainaccord4', 'mainaccord5'],
      dtype='object')

In [ ]:
print(f"Shape: {df.shape}")  # (rows, columns)

Shape: (24063, 13)


In [ ]:
df.head(10)

,Unnamed: 0,url,Perfume,Brand,Rating Count,Top,Middle,Base,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5
0,0,https://www.fragrantica.com/perfume/xerjoff/ac...,Accento Overdose Pride Edition,Xerjoff,201,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",rose,woody,fruity,aromatic,floral
1,1,https://www.fragrantica.com/perfume/jean-paul-...,Classique Pride 2024,Jean Paul Gaultier,70,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",citrus,white floral,sweet,fresh,musky
2,2,https://www.fragrantica.com/perfume/jean-paul-...,Classique Pride 2023,Jean Paul Gaultier,285,"blood orange, yuzu","neroli, orange blossom","musk, white woods",citrus,white floral,sweet,fresh spicy,musky
3,3,https://www.fragrantica.com/perfume/bruno-bana...,Pride Edition Man,Bruno Banani,59,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",fruity,nutty,woody,tropical,NaN
4,4,https://www.fragrantica.com/perfume/jean-paul-...,Le MÃ¢le Pride Collector,Jean Paul Gaultier,632,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",aromatic,warm spicy,fresh spicy,cinnamon,vanilla
5,5,https://www.fragrantica.com/perfume/jean-paul-...,Le Male Pride 2023,Jean Paul Gaultier,531,"yuzu, blood orange","neroli, orange blossom","musk, white woods",citrus,white floral,sweet,fresh spicy,green
6,6,https://www.fragrantica.com/perfume/jean-paul-...,Le Male Pride 2024,Jean Paul Gaultier,285,"citruses, yuzu","neroli, orange blossom","woodsy notes, musk",white floral,citrus,fresh,soapy,NaN
7,7,https://www.fragrantica.com/perfume/ralph-laur...,Polo Red Pride Edition,Ralph Lauren,107,"cranberry, pink grapefruit, citron","clary sage, lavender, saffron","amberwood, brazilian redwood, coffee",woody,amber,citrus,warm spicy,aromatic
8,8,https://www.fragrantica.com/perfume/ralph-laur...,Ralph Pride Edition,Ralph Lauren,50,"tangerine, green apple, freesia","magnolia, lime (linden blossom), osmanthus","musk, white iris",citrus,fruity,floral,sweet,green
9,9,https://www.fragrantica.com/perfume/we-pink/wa...,Waffle,We Pink,27,"strawberry, almond, raspberry","butter, peach blossom, violet","praline, vanilla, musk",powdery,musky,violet,fruity,fresh


# Load Data (with Reviews)


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/totallymakescents/perfumes_table.csv'
with open(file_path, encoding='latin1') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        print(f"Line {i+1}: {row}")
        if i >= 9:  # print first 10 lines
            break

Line 1: ['rating', 'notes', 'designer', 'reviews', 'description', 'url', 'title']
Line 2: ['5.0', "['Vanila', 'Madagascar Vanilla', 'Vanilla Absolute']", 'fiorucci perfumes and colognes', '[]', 'Vanilla Scent by Fiorucci is a Amber Vanilla fragrance for women and men. Vanilla Scent was launched in 1961. Top note is Vanila; middle note is Madagascar Vanilla; base note is Vanilla Absolute.', 'https://www.fragrantica.com/perfume/Fiorucci/Vanilla-Scent-34262.html', 'Vanilla Scent Fiorucci for women and men']
Line 3: ['4.22', "['Mint', 'Lavender', 'Nutmeg', 'elemi', 'Cypress', 'Sandalwood', 'Saffron', 'Cypriol Oil or Nagarmotha', 'Amber', 'Patchouli', 'Precious Woods', 'Musk']", 'maiora parfum perfumes and colognes', '[]', 'Shady by Maiora Parfum is a fragrance for women and men. Shady was launched in 2019. The nose behind this fragrance is Antonio Gigli. Top notes are Mint, Lavender, Nutmeg and elemi; middle notes are Cypress, Sandalwood, Saffron and Cypriol Oil or Nagarmotha; base notes a

In [ ]:
df_review = pd.read_csv(
    file_path,
    sep=',',            # specify correct separator
    encoding='latin1',
    low_memory=False
)

In [ ]:
df_review.columns

Index(['rating', 'notes', 'designer', 'reviews', 'description', 'url',
       'title'],
      dtype='object')

In [ ]:
print(f"Shape: {df_review.shape}")  # (rows, columns)

Shape: (84144, 7)


In [ ]:
df_review.head(10)

,rating,notes,designer,reviews,description,url,title
0,5.00,"['Vanila', 'Madagascar Vanilla', 'Vanilla Abso...",fiorucci perfumes and colognes,[],Vanilla Scent by Fiorucci is a Amber Vanilla f...,https://www.fragrantica.com/perfume/Fiorucci/V...,Vanilla Scent Fiorucci for women and men
1,4.22,"['Mint', 'Lavender', 'Nutmeg', 'elemi', 'Cypre...",maiora parfum perfumes and colognes,[],Shady by Maiora Parfum is a fragrance for wome...,https://www.fragrantica.com/perfume/Maiora-Par...,Shady Maiora Parfum for women and men
2,3.81,"['Neroli', 'Vetiver', 'Basil', 'Fig']",guerlain perfumes and colognes,"[""the opening is too sharp that it reminds me ...",Nerolia Vetiver by Guerlain is a Woody Floral ...,https://www.fragrantica.com/perfume/Guerlain/N...,Nerolia Vetiver Guerlain for women and men
3,4.00,"['Orange Blossom', 'Star Anise', 'Pear', 'Rose...",jean paul gaultier perfumes and colognes,['I must ask. What part of the country did thi...,Classique Love Actually by Jean Paul Gaultier ...,https://www.fragrantica.com/perfume/Jean-Paul-...,Classique Love Actually Jean Paul Gaultier for...
4,4.22,"['Petitgrain', 'Orange', 'Bergamot', 'Lemon Ve...",santa maria novella perfumes and colognes,['Beautiful and uplifting. Initially fresh and...,Zagara (Orange Blossom) by Santa Maria Novella...,https://www.fragrantica.com/perfume/Santa-Mari...,Zagara (Orange Blossom) Santa Maria Novella fo...
5,3.66,"['Bamboo', 'Violet', 'Water Hyacinth', 'Kiwi',...",oriflame perfumes and colognes,['DO starts airy and fresh proceeding quickly ...,Divine by Oriflame is a Floral fragrance for w...,https://www.fragrantica.com/perfume/Oriflame/D...,Divine Oriflame for women
6,NaN,"['Lemon', 'Cardamom', 'Lily-of-the-Valley', 'A...",jequiti perfumes and colognes,[],Tiago Abravanel Mania by Jequiti is a Woody Ar...,https://www.fragrantica.com/perfume/Jequiti/Ti...,Tiago Abravanel Mania Jequiti for men
7,3.74,"['Cherry', 'Mandarin Orange', 'Datura', 'Ylang...",panouge perfumes and colognes,['Not sure what it is but this smells like a w...,Datura Amaretti by Panouge is a Amber Floral f...,https://www.fragrantica.com/perfume/Panouge/Da...,Datura Amaretti Panouge for women and men
8,2.33,"['Mandarin Orange', 'Freesia', 'Orange Blossom...",franck olivier perfumes and colognes,['Smells like a multisex version of Intimately...,Baby Boy by Franck Olivier is a Citrus fragran...,https://www.fragrantica.com/perfume/Franck-Oli...,Baby Boy Franck Olivier for men
9,4.33,"['Hawthorn', 'Iris', 'Iris', 'Tuberose', 'Tiar...",dâantolâ parfums perfumes and colognes,[],Theatre â Ð¢ÐµÐ°ÑÑ by DâAntolâ Parfums...,https://www.fragrantica.com/perfume/D-Antol-Pa...,Theatre â Ð¢ÐµÐ°ÑÑ DâAntolâ Parfums fo...


# Merge Data

In [ ]:
df['url'] = df['url'].str.strip().str.lower()
df_review['url'] = df_review['url'].str.strip().str.lower()

# remove trailing "/"
df['url'] = df['url'].str.rstrip('/')
df_review['url'] = df_review['url'].str.rstrip('/')

In [ ]:
# concatinate all non-nan reviews into a single string
agg_reviews = df_review.groupby('url')['reviews'].apply(lambda x: ' '.join(x.dropna().astype(str))).reset_index()

combined_df = pd.merge(df, agg_reviews, on='url', how='left')

In [ ]:
print(combined_df.columns)
print(combined_df.shape)
combined_df.head()

Index(['Unnamed: 0', 'url', 'Perfume', 'Brand', 'Rating Count', 'Top',
       'Middle', 'Base', 'mainaccord1', 'mainaccord2', 'mainaccord3',
       'mainaccord4', 'mainaccord5', 'scent_description', 'reviews'],
      dtype='object')
(24063, 15)


,Unnamed: 0,url,Perfume,Brand,Rating Count,Top,Middle,Base,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5,scent_description,reviews
0,0,https://www.fragrantica.com/perfume/xerjoff/ac...,Accento Overdose Pride Edition,Xerjoff,201,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",rose,woody,fruity,aromatic,floral,"fruity notes, aldehydes, green notes bulgarian...","[""I thought everybody was hating on this fragr..."
1,1,https://www.fragrantica.com/perfume/jean-paul-...,Classique Pride 2024,Jean Paul Gaultier,70,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",citrus,white floral,sweet,fresh,musky,"yuzu, citruses orange blossom, neroli musk, bl...",NaN
2,2,https://www.fragrantica.com/perfume/jean-paul-...,Classique Pride 2023,Jean Paul Gaultier,285,"blood orange, yuzu","neroli, orange blossom","musk, white woods",citrus,white floral,sweet,fresh spicy,musky,"blood orange, yuzu neroli, orange blossom musk...","['Ummm... just like a honest review, ignoring ..."
3,3,https://www.fragrantica.com/perfume/bruno-bana...,Pride Edition Man,Bruno Banani,59,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",fruity,nutty,woody,tropical,NaN,"guarana, grapefruit, red apple walnut, lavende...",[]
4,4,https://www.fragrantica.com/perfume/jean-paul-...,Le MÃ¢le Pride Collector,Jean Paul Gaultier,632,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",aromatic,warm spicy,fresh spicy,cinnamon,vanilla,"mint, lavender, cardamom, artemisia, bergamot ...","['Lol to the reviewer below meâ¦no one cares,..."


In [ ]:
non_null_reviews = combined_df['reviews'].notnull().sum()
print(f"Perfumes with reviews: {non_null_reviews} / {len(combined_df)}")

Perfumes with reviews: 23290 / 24063


# Sentence-BERT

In [ ]:
pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:
# load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# select note columns
note_cols = ['Top', 'Middle', 'Base',
             'mainaccord1', 'mainaccord2', 'mainaccord3',
             'mainaccord4', 'mainaccord5']

# fill NaNs and combine into a string
# df['scent_description'] = df[note_cols].fillna('').agg(' '.join, axis=1)

In [ ]:
combined_df['reviews'] = combined_df['reviews'].str.replace(r'^\[?"|"?\]$', '', regex=True)

In [ ]:
# def build_full_description(row):
#     scent_text = ' '.join([str(row.get(col, '')) for col in note_cols])
#     review_text = str(row.get('reviews', ''))
#     return (scent_text + ' ' + review_text).strip()

def build_full_description(row):
    # so that BERT can understand different fields

    top = row.get('Top', '')
    middle = row.get('Middle', '')
    base = row.get('Base', '')

    accords = ', '.join([
        str(row.get('mainaccord1', '')),
        str(row.get('mainaccord2', '')),
        str(row.get('mainaccord3', '')),
        str(row.get('mainaccord4', '')),
        str(row.get('mainaccord5', ''))
    ])

    reviews = row.get('reviews', '')

    description = (
        f"Top Notes: {top}. "
        f"Middle Notes: {middle}. "
        f"Base Notes: {base}. "
        f"Main Accords: {accords}. "
        f"User Review: {reviews}"
    )

    return description.strip()

combined_df['full_description'] = combined_df.apply(build_full_description, axis=1)

In [ ]:
combined_df.columns

Index(['Unnamed: 0', 'url', 'Perfume', 'Brand', 'Rating Count', 'Top',
       'Middle', 'Base', 'mainaccord1', 'mainaccord2', 'mainaccord3',
       'mainaccord4', 'mainaccord5', 'scent_description', 'reviews',
       'full_description'],
      dtype='object')

In [ ]:
# see the change
combined_df[['Top', 'Middle', 'Base',
    'mainaccord1', 'mainaccord2', 'mainaccord3',
    'mainaccord4', 'mainaccord5', 'full_description']].head(5)

,Top,Middle,Base,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5,full_description
0,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",rose,woody,fruity,aromatic,floral,"Top Notes: fruity notes, aldehydes, green note..."
1,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",citrus,white floral,sweet,fresh,musky,"Top Notes: yuzu, citruses. Middle Notes: orang..."
2,"blood orange, yuzu","neroli, orange blossom","musk, white woods",citrus,white floral,sweet,fresh spicy,musky,"Top Notes: blood orange, yuzu. Middle Notes: n..."
3,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",fruity,nutty,woody,tropical,NaN,"Top Notes: guarana, grapefruit, red apple. Mid..."
4,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",aromatic,warm spicy,fresh spicy,cinnamon,vanilla,"Top Notes: mint, lavender, cardamom, artemisia..."


In [ ]:
print(combined_df['full_description'].iloc[0])

Top Notes: fruity notes, aldehydes, green notes. Middle Notes: bulgarian rose, egyptian jasmine, lily-of-the-valley. Base Notes: eucalyptus, pine. Main Accords: rose, woody, fruity, aromatic, floral. User Review: I thought everybody was hating on this fragrance for the homophobia, turns out its a cash grab where none of the profits go to any lgbtq organisation. don't buy", "Above and beyond super yikes totally gross pink-washing money grab. Next time give 50-100% of profits from the sales to LGBTQ+ orgs, AT THE VERY LEAST, especially since y'all are selling it at that price. Please Xerjoff, spare some change for the gays.", 'Embarrassing pink money cash grab, and during Pride month nonetheless! Xerjoff needs to show us where they\'re donating because simply saying they "support" the LGBTQ community is not enough...', 'FYI: When asked if some LGBTQ+ organization would receive part of earning from this bottle, the PR team of Xerjoff decided to copy paste to everyone a answer without answ

In [ ]:
# now we are ready to build embeddings

scent_lists_with_review = combined_df['full_description'].tolist()
scent_embeddings = model.encode(
    scent_lists_with_review,
    batch_size=32,
    convert_to_tensor=True,
    show_progress_bar=True
)

Batches:   0%|          | 0/752 [00:00<?, ?it/s]

In [ ]:
def recommend_perfumes(user_query, top_k=5):
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    similarities = util.cos_sim(query_embedding, scent_embeddings)[0]
    # print(similarities)
    top_results = torch.topk(similarities, k=top_k)

    print(f"\nUser Query: {user_query}\n")
    for score, idx in zip(top_results.values, top_results.indices):
        idx = idx.item()        # convert Python tensor to int
        perfume = df.iloc[idx]
        print(f"{perfume['Perfume']} by {perfume['Brand']} (Score: {score.item():.3f})")
        print(f"Notes: {perfume['scent_description']}\n")

In [ ]:
recommend_perfumes("A magical scent in a mystical forest with herbs and secrets")


User Query: a magical scent in a mystical forest with herbs and secrets

Ecstasy by Tiziana Terenzi (Score: 0.662)
Notes: spruce, pine tree, pebbles incense, rose, patchouli, violet soil tincture, amber, woody notes, labdanum, sandalwood, tonka bean woody amber aromatic balsamic smoky

Foreste di Seta by Salvatore Ferragamo (Score: 0.650)
Notes: clary sage walnut, silk madagascar vetiver aromatic woody soft spicy earthy nutty

La Foret Russe by Nimere Parfums (Score: 0.649)
Notes: thyme, sage, herbal notes siberian pine, cypress gurjan balsam, incense, cedar, oakmoss, vetiver, benzoin woody aromatic green fresh spicy amber

Moncler pour Homme by Moncler (Score: 0.644)
Notes: green notes, clary sage pine sandalwood, cedar, vetiver, amber woody aromatic green powdery 

SÃ¨ve Nouvelle by Thomas Kosmala (Score: 0.643)
Notes: saffron, pink pepper, grapefruit champaca, water pepper, tonka bean cashmirwood, patchouli, labdanum, amberwood warm spicy woody amber musky floral



In [ ]:
recommend_perfumes("What’s it smell like in the rain, at the end of a hiking trail full of blossoms?")


User Query: What’s it smell like in the rain, at the end of a hiking trail full of blossoms?

Steamed Rainbow by DS&Durga (Score: 0.629)
Notes: blood mandarin, orange, elemi grass, almond blossom, cedar vetiver, violet citrus green woody aromatic floral

French Affair by Ex Nihilo (Score: 0.598)
Notes: litchi, violet leaf, bergamot rose oil, atlas cedar, angelica oakmoss, vetiver, patchouli woody earthy mossy aromatic rose

Monsoon by Dame Perfumery (Score: 0.579)
Notes: orris root, iris flower water lily, lily-of-the-valley, jasmine creosote bush, cedar iris woody powdery white floral floral

La 13Ã¨me Note Femme by Absolument Parfumeur (Score: 0.573)
Notes: sage, pineapple, raspberry, wild strawberry mimosa, vanilla, jasmine, violet, rose white peach, musk, honey, amber sweet fruity powdery floral yellow floral

Daisy Skies by Marc Jacobs (Score: 0.570)
Notes: mineral notes, violet leaves, quince lotus, violet, jasmine seaweed, musk, sandalwood aquatic ozonic floral powdery marine



In [ ]:
recommend_perfumes("What fragrance would a wizard wear in a magical world?")


User Query: What fragrance would a wizard wear in a magical world?

LP No. 9 by Penhaligon's (Score: 0.607)
Notes: tarragon, geranium, lavender, bergamot, amalfi lemon carnation, rose, jasmine virginia cedar, cinnamon, amber, patchouli, musk, vanille aromatic warm spicy fresh spicy floral woody

White Diamonds Parfum by Elizabeth Taylor (Score: 0.593)
Notes: aldehydes, white lily, orange, neroli, bergamot ylang-ylang, narcissus, tuberose, jasmine, orris root, cinnamon, turkish rose, cloves sandalwood, amber, oakmoss, musk, patchouli white floral woody yellow floral warm spicy citrus

Baudelaire by Byredo (Score: 0.588)
Notes: juniper berries, pepper, caraway leather, incense, hyacinth patchouli, papyrus, amber fresh spicy woody amber aromatic leather

Eau de Minuit - Midnight Fragrance by Lolita Lempicka (Score: 0.584)
Notes: myrrh, anise, ivy violet, iris, jasmine vanilla, resin, tonka bean amber vanilla powdery violet sweet

Tone Indeterminee by Zara (Score: 0.582)
Notes: cinnamon, 

In [ ]:
recommend_perfumes("Looking for a bittersweet scent for a farewell party.")


User Query: Looking for a bittersweet scent for a farewell party.

Reverence by Princesse Marina De Bourbon (Score: 0.652)
Notes: bergamot, spicy notes, pepper jasmine, plum, fruity notes, rose, tea musk, sandalwood fruity white floral sweet citrus fresh spicy

Versace Pour Femme by Versace (Score: 0.642)
Notes: lilac, guava, wisteria, black currant, dew drop jasmine, lotus, orchid, rhododendron musk, cashmere wood, bourbon vetiver, atlas cedar floral fresh fruity white floral woody

She by Revlon (Score: 0.638)
Notes: champagne, grapefruit, ginger, mandarin orange honeysuckle, rose, cyclamen, cardamom, magnolia musk, woodsy notes, sandalwood citrus warm spicy champagne floral aldehydic

Tone Indeterminee by Zara (Score: 0.637)
Notes: cinnamon, pomelo neroli tobacco cinnamon tobacco warm spicy citrus sweet

La Commedia (2017) by Salvatore Ferragamo (Score: 0.636)
Notes: cardamom, bitter orange pimento, nutmeg guaiac wood, vetiver warm spicy woody aromatic fresh spicy fresh



# LLM Generates Explanation
Follows this notebook:
https://colab.research.google.com/drive/1T5-zKWM_5OD21QHwXHiV9ixTRR7k3iB9?usp=sharing

In [ ]:
# Chat-GPT generated data, will replace later with data from Hugging Face
# to train LLM to generate meaningful explanation, need more data ~100-200, will add more
in_out_data = [
    {
    "input": "User wants a scent for a farewell party. Scent: citrus, violet, ambergris.",
    "output": "This perfume evokes a bittersweet goodbye — bright citrus for the goodbye, softened by violet and grounded in ambergris like a fading memory."
    },
    {
    "input": "User seeks a scent for a first date. Scent: raspberry, rose, sandalwood.",
    "output": "Flirty and romantic — sweet raspberry teases at the top, while rose adds elegance, and sandalwood brings warm confidence."
    },
    {
    "input": "User asks for a scent that feels like winter morning in a snowy cabin. Scent: pine, incense, vanilla.",
    "output": "This scent blends cool pine with smoky incense and cozy vanilla — a quiet snow-covered morning by the fire."
    },
    {
    "input": "User wants something that smells like a library filled with old books. Scent: leather, cedarwood, patchouli.",
    "output": "Evoking antique leather and aged wood, this scent conjures dusty pages, quiet corners, and timeless thought."
    },
    {
      "input": "User needs a scent for celebrating a big achievement. Scent: champagne accord, white florals, musk.",
      "output": "A sparkling celebration — fizzy champagne and bright florals rise with joy, grounded by elegant musk."
    }
]

In [ ]:
import json

with open("perfume_reasoning_dataset.jsonl", "w") as f:
    for example in in_out_data:
        json_line = json.dumps(example)
        f.write(json_line + "\n")


## Install Unsloth

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Data Prep

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="perfume_reasoning_dataset.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# needs this format when fine-tuning LLaMA3
def formatting_prompts_func(example):
    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{example['input']}<|eot_id|>\n" \
             f"<|start_header_id|>assistant<|end_header_id|>\n\n{example['output']}<|eot_id|>"
    return {"text": prompt}

dataset = dataset.map(formatting_prompts_func)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
print(dataset[4]["text"])

<|start_header_id|>user<|end_header_id|>

User needs a scent for celebrating a big achievement. Scent: champagne accord, white florals, musk.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

A sparkling celebration — fizzy champagne and bright florals rise with joy, grounded by elegant musk.<|eot_id|>


## Finetune Model with LLama3.2-Unsloth

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
# Only training on assistant responses

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
# Verifying data format

tokenizer.decode(trainer.train_dataset[4]["input_ids"])

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nUser needs a scent for celebrating a big achievement. Scent: champagne accord, white florals, musk.<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\nA sparkling celebration — fizzy champagne and bright florals rise with joy, grounded by elegant musk.<|eot_id|>'

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[4]["labels"]])

'                                  A sparkling celebration — fizzy champagne and bright florals rise with joy, grounded by elegant musk.<|eot_id|>'

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.441 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 60 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,3.883200
2,3.884000
3,3.816600
4,3.558900
5,3.188000
6,2.760400
7,2.314400
8,1.933100
9,1.560200
10,1.179000


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

78.0656 seconds used for training.
1.3 minutes used for training.
Peak reserved memory = 3.441 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 23.343 %.
Peak reserved memory for training % of max memory = 0.0 %.


## Run the model

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {
        "role": "user",
        "content": "User wants a scent for a farewell party. Scent: citrus, violet, ambergris.",
    },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nUser wants a scent for a farewell party. Scent: citrus, violet, ambergris.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThis perfume evokes a bittersweet goodbye — bright citrus for the goodbye, softened by violet and grounded in ambergris like a fading memory.<|eot_id|>']

In [ ]:
# Use a TextStreamer for continuous inference

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {
        "role": "user",
        "content": "User wants a scent for a graduation ceremony. Scent: citrus, violet, ambergris.",
    },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

This perfume evokes a bittersweet moment — bright citrus for the celebration, softened by violet and grounded in ambergris like a fading memory.<|eot_id|>


In [ ]:
# overfitting because only has 5 data set in the in_out_data, needs to add more data
# see a way for users to input the next prompt
# use the output from sentence-bert as an input for LLM